In [1]:
import ROOT
data_file_electrons = ROOT.TFile.Open("http://opendata.atlas.cern/release/samples/Data/DataEgamma.root")
tree_electrons = data_file_electrons.Get("mini")

Welcome to JupyROOT 6.14/04


In [2]:
def getInvMass(lep1_pt, lep1_eta, lep1_phi, lep2_pt, lep2_eta, lep2_phi):
    ''' get invariant mass '''
    import math
    msq = 2*lep1_pt*lep2_pt*(math.cosh(lep1_eta-lep2_eta) - math.cos(lep1_phi-lep2_phi))
    return math.sqrt(msq)

In [3]:
# histos
nbins=50; lowedge=30.; upedge=140.
h_mass_electrons = ROOT.TH1F("h_mass_electrons", "; Invariant mass [GeV]; Number of events", nbins, lowedge, upedge)

### event loop

In [4]:
ievt = 0
for evt in tree_electrons:
    if evt.lep_n != 2: continue
    if not (evt.lep_type[0]==11 and evt.lep_type[1]==11): continue
    if (evt.lep_charge[0] + evt.lep_charge[1]) != 0: continue 
    if ievt > 4000:
        break

    h_mass_electrons.Fill(getInvMass(evt.lep_pt[0]*1e-3, evt.lep_eta[0], evt.lep_phi[0], evt.lep_pt[1]*1e-3, evt.lep_eta[1], evt.lep_phi[1]))
    
    ievt += 1

### Draw

In [5]:
%jsroot on

In [6]:
canvas = ROOT.TCanvas("canvas", "", 800, 600)

In [7]:
#### Prepare: setting Breit-Wigner limits, colors on functions etc.
norm_low = 100; norm_up = 15e3
mean_low = 80; mean_up = 100
width_low = 0.1; width_up = 40
def setBWLims(func):
    ''' Set limits on BW parameters '''
    func.SetParameters(0.5*(norm_low+norm_up), 0.5*(mean_low+mean_up), 0.5*(width_low+width_up))
    func.SetParLimits(0, norm_low, norm_up)
    func.SetParLimits(1, mean_low, mean_up)
    func.SetParLimits(2, width_low, width_up)
def setColor(ROOT_obj, color=ROOT.kBlack):
    ''' set (line) color'''
    ROOT_obj.SetLineColor(color)

In [8]:
#### DEFINE SOME BREIT-WIGNER FUNCTIONS
xlow_fit=40.; xup_fit=140.;
#foo=ROOT.TMath.BreitWigner(4., 3., 3.)
fBW = ROOT.TF1("BW", "[0]*TMath::BreitWigner(x, [1], [2])", xlow_fit, xup_fit)
setBWLims(fBW)
# constant plus exponential for bkg
fBkg = ROOT.TF1("bkg", "pol0+expo(1)", xlow_fit, xup_fit)
fModel = ROOT.TF1("model", "BW+bkg", xlow_fit, xup_fit)
# set color
setColor(fModel); 
setColor(fBW, ROOT.kRed)
setColor(fBkg, ROOT.kBlue)

### Fit

In [9]:
fitresult=h_mass_electrons.Fit("model", "S", "e1p", xlow_fit, xup_fit) # store fit result
## draw signal and bkg separately
fitresult_parameters={}
for i_p,p in enumerate(fitresult.Parameters()):
    fitresult_parameters[i_p]=(p, fitresult.ParError(i_p))
### signal
for ipar in range(3): fBW.SetParameter(ipar, fitresult_parameters[ipar][0])
### bkg
for ipar in range(2): fBkg.SetParameter(ipar, fitresult_parameters[ipar+3][0])
### DRAW

# test: some text
text=ROOT.TLatex()
text.DrawLatexNDC(0.4,0.4,"foobar")

# fBkg.Draw("SAME")
fBW.Draw("SAME") ## COMMENTING THIS OUT MAKES THE PLOT INTERACTIVE AS IS SHOULD

canvas.Draw()
canvas.SaveAs("./jsproblem.png") ## THE SAVED PLOT HAS THE SIGNAL DRAWN


 FCN=163.286 FROM MIGRAD    STATUS=CONVERGED     278 CALLS         279 TOTAL
                     EDM=4.20364e-09    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   0.4 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           7.19077e+03   1.47008e+02  -5.20198e-03   1.85096e-07
   2  p1           8.98065e+01   7.43418e-02   4.97521e-06   1.65911e-04
   3  p2           5.34336e+00   1.49240e-01  -3.81839e-06   1.68512e-04
   4  p3          -4.24210e+00   1.19641e+00  -4.93930e-04  -2.99218e-05
   5  p4           4.34920e+00   3.28278e-01   3.31918e-05  -5.77423e-04
   6  p5          -3.04182e-02   7.52706e-03  -3.46329e-07  -4.11128e-02


Info in <TCanvas::Print>: png file ./jsproblem.png has been created
